In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from pyts.image import GramianAngularField

from sklearn.preprocessing import StandardScaler
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import h5py

In [2]:
## Load the data
cloudburst_data = np.load('CB12.npy')
non_cloudburst_data = np.load('NB.npy')

print(cloudburst_data.shape)
print(non_cloudburst_data.shape)

# Combine the cloudburst and non-cloudburst data into one array
X = np.concatenate((cloudburst_data, non_cloudburst_data))

# Create a target vector (1 for cloudburst, 0 for non-cloudburst)
y = np.concatenate((np.ones(cloudburst_data.shape[0]), np.zeros(non_cloudburst_data.shape[0])))

# Shuffle the data
shuffle_index = np.random.permutation(X.shape[0])
X = X[shuffle_index]
y = y[shuffle_index]

print(X.shape)
print(y.shape)

(17, 6, 256, 256)
(96, 6, 256, 256)
(113, 6, 256, 256)
(113,)


In [4]:
optimizers = ['sgd']
kernel_sizes = [(1, 1), (2, 2), (3, 3)]
pooling_sizes = [(1, 1), (2, 2), (3, 3)]
learning_rates = [0.0001, 0.0005, 0.001]

for i, opt in enumerate(optimizers):
    for j, k_size in enumerate(kernel_sizes):
        for l, p_size in enumerate(pooling_sizes):
            for m, lr in enumerate(learning_rates):
                # Get the current TensorFlow session
                sess = tf.compat.v1.get_default_session()

                # Close the session to free up the GPU memory
                if sess is not None:
                    sess.close()

                # Reset the TensorFlow graph
                tf.compat.v1.reset_default_graph()

                
                try:
                    # Create a new model for each hyperparameter combination
                    model = Sequential(name=f'sequential_{i}_{j}_{l}_{m}')
                    model.add(InputLayer(input_shape=(6, 256, 256)))
                    model.add(Conv2D(filters=128, kernel_size=k_size, activation='relu', padding='same'))
                    model.add(MaxPooling2D(pool_size=p_size, padding='same'))
                    model.add(Conv2D(filters=256, kernel_size=k_size, activation='relu', padding='same'))
                    model.add(MaxPooling2D(pool_size=(1, 1), padding='same'))
                    model.add(Conv2D(filters=512, kernel_size=k_size, activation='relu', padding='same'))
                    model.add(MaxPooling2D(pool_size=p_size, padding='same'))
                    model.add(Flatten())
                    model.add(Dense(units=256, activation='relu', kernel_regularizer='l1_l2'))
                    model.add(Dropout(rate=0.5))
                    model.add(Dense(units=1, activation='sigmoid'))

                    optimizer = None
                    if opt == 'adam':
                        optimizer = Adam(learning_rate=lr)
                    elif opt == 'sgd':
                        optimizer = SGD(learning_rate=lr)
                    elif opt == 'rmsprop':
                        optimizer = RMSprop(learning_rate=lr)

                    # Compile the model with binary cross-entropy loss function
                    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

                    # Train the model on your data with 50 epochs and batch size of 1
                    history = model.fit(X, y, epochs=50, batch_size=1, validation_split=0.3)

                    # Print the output of each combination
                    print(f"Optimizer: {opt}, Kernel Size: {k_size}, Pooling Size: {p_size}, Learning Rate: {lr}")
                    print(f"Training accuracy: {history.history['accuracy'][-1]}, Validation accuracy: {history.history['val_accuracy'][-1]}")
                    print("-" * 50)

                    # Calculate and print the confusion matrix
                    y_pred = np.round(model.predict(X)).flatten()
                    y_true = y.flatten()
                    cm = confusion_matrix(y_true, y_pred)
                    print("Confusion Matrix:")
                    print(cm)

                    accuracy = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
                    print("Accuracy:", accuracy)

                    test = np.load('D:\\TEST12.npy')
                    print(test.shape)
                    prediction = model.predict(test)

                    # Multiply the output by 100 and convert it to integers
                    prediction_int = (prediction * 100).astype(int)

                    # Print the result
                    print(prediction_int)

                    
                    model.save(f'D:\\Models\\sequential_{i}_{j}_{l}_{m}.h5')

                except Exception as e:
                    # If there's an error, print a message and continue to the next combination
                    print(f"Error for hyperparameter combination optimizer={opt}, kernel_size={k_size}, pooling_size={p_size}, learning_rate={lr}: {e}")
                    continue


Epoch 1/50
79/79 [==============================] - 48s 594ms/step - loss: 2707.3738 - accuracy: 0.7722 - val_loss: 2627.8445 - val_accuracy: 0.8824
Epoch 2/50
79/79 [==============================] - 34s 436ms/step - loss: 2551.9939 - accuracy: 0.8101 - val_loss: 2474.7920 - val_accuracy: 0.8824
Epoch 3/50
79/79 [==============================] - 34s 435ms/step - loss: 2401.2346 - accuracy: 0.8101 - val_loss: 2326.3962 - val_accuracy: 0.8824
Epoch 4/50
79/79 [==============================] - 35s 438ms/step - loss: 2255.0806 - accuracy: 0.8481 - val_loss: 2182.5959 - val_accuracy: 0.8824
Epoch 5/50
79/79 [==============================] - 34s 430ms/step - loss: 2113.5317 - accuracy: 0.8228 - val_loss: 2043.3925 - val_accuracy: 0.8824
Epoch 6/50
79/79 [==============================] - 33s 412ms/step - loss: 1976.6057 - accuracy: 0.8481 - val_loss: 1908.8135 - val_accuracy: 0.8824
Epoch 7/50
79/79 [==============================] - 35s 439ms/step - loss: 1844.2257 - accuracy: 0.8608 - 

In [5]:
# Calculate and print the confusion matrix
y_pred = np.round(model.predict(X)).flatten()
y_true = y.flatten()
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

accuracy = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
print("Accuracy:", accuracy)

4/4 [==============================] - 0s 72ms/step
Confusion Matrix:
[[95  1]
 [ 2 15]]
Accuracy: 0.9734513274336283


In [ ]:
import h5py
model.save('latest_6_better2.h5')

In [ ]:
from keras.models import load_model

model = load_model('latest_6_better.h5')

In [7]:
test = np.load('D:\\TEST6.npy')
print(test.shape)
prediction = model.predict(test)

# Multiply the output by 100 and convert it to integers
prediction_int = (prediction * 100).astype(int)

# Print the result
print(prediction_int)


(5, 6, 256, 256)
1/1 [==============================] - 0s 27ms/step
[[ 5]
 [17]
 [ 0]
 [ 0]
 [ 0]]
